Make data and content

In [50]:
from langchain.document_loaders import WebBaseLoader
 
#add website data
URL = ["https://www.geeksforgeeks.org/stock-price-prediction-project-using-tensorflow/",
       "https://www.geeksforgeeks.org/training-of-recurrent-neural-networks-rnn-in-tensorflow/"]
 
#load the data
data = WebBaseLoader(URL)
#extract the content
content = data.load()

split large doc into a smaller doc

In [51]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
 
# a langchain object
text_splitter = RecursiveCharacterTextSplitter(chunk_size=256,chunk_overlap=50)
# a list of dictionaries containing info like source, title, description, and page_content
chunking = text_splitter.split_documents(content) 


In [52]:
aux_array = list(map(lambda page: page.page_content, chunking))
texts = text_splitter.create_documents(aux_array)

In [53]:
text_splitter

Use Hugging Face embeddings API to generate embeddings, which are meant to convert text into vectors

In [54]:
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings
 
import os
from getpass import getpass
 
# get your free access token from HuggingFace and paste it here
HF_token = getpass()
os.environ['HUGGINGFACEHUB_API_TOKEN'] = HF_token
 
embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key = HF_token,model_name = "BAAI/bge-base-en-v1.5"
)

In [55]:
embeddings

HuggingFaceInferenceAPIEmbeddings(api_key=SecretStr('**********'), model_name='BAAI/bge-base-en-v1.5', api_url=None, additional_headers={})

Create a vector store to store the embeddings and retrieve later

In [19]:
?Chroma.from_documents

Signature:
Chroma.from_documents(
    documents: 'List[Document]',
    embedding: 'Optional[Embeddings]' = None,
    ids: 'Optional[List[str]]' = None,
    collection_name: 'str' = 'langchain',
    persist_directory: 'Optional[str]' = None,
    client_settings: 'Optional[chromadb.config.Settings]' = None,
    client: 'Optional[chromadb.Client]' = None,
    collection_metadata: 'Optional[Dict]' = None,
    **kwargs: 'Any',
) -> 'Chroma'
Docstring:
Create a Chroma vectorstore from a list of documents.

If a persist_directory is specified, the collection will be persisted there.
Otherwise, the data will be ephemeral in-memory.

Args:
    collection_name (str): Name of the collection to create.
    persist_directory (Optional[str]): Directory to persist the collection.
    ids (Optional[List[str]]): List of document IDs. Defaults to None.
    documents (List[Document]): List of documents to add to the vectorstore.
    embedding (Optional[Embeddings]): Embedding function. Defaults to None.


In [34]:
pgs = [chunking[i].page_content for i in range(len(chunking))]
pgs[0]

'Stock Price Prediction Project using TensorFlow - GeeksforGeeks\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content'

In [56]:
from langchain.vectorstores import Chroma


vectorstore = Chroma("embedding_db", embeddings)
db= vectorstore.from_documents(texts, embedding=embeddings)

KeyError: 0

In [ ]:
retriever = vectorstore.as_retriever(search_type="mmr",search_kwargs={"k":3})
query = "what is recurrent neural network?"
docs_rel = retriever.get_relevant_documents(query)
print(docs_rel)

NameError: name 'vectorstore' is not defined

In [ ]:
prompt = f"""
<|system|>>
You are an AI Assistant that follows instructions extremely well.
Please be truthful and give direct answers. Please tell 'I don't know' if user query is not in context
</s>
<|user|>
{query}
</s>
<|assistant|>
"""